In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv('cleaned_data/NormalizedCountsMetadata.csv')
df.head()

,Unnamed: 0,SampleID,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000284735,ENSG00000284737,ENSG00000284738,ENSG00000284740,ENSG00000284743,ENSG00000284747,ENSG00000284748,SampleTiming,FrailtyStatus,Age
0,0,0029-0,7.0,31.0,7.0,385.0,3.0,12.0,30.0,39.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,pre-vaccination,frail,73.6
1,1,0029-3,7.0,37.0,10.0,400.0,3.0,15.0,34.0,46.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3 days post-vaccination,frail,73.6
2,2,0029-7,5.0,28.0,9.0,485.0,3.0,13.0,31.0,39.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7 days post-vaccination,frail,73.6
3,3,0104-0,11.0,47.0,10.0,320.0,4.0,14.0,30.0,63.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,pre-vaccination,non-frail,62.2
4,4,0104-3,9.0,36.0,10.0,203.0,3.0,13.0,47.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3 days post-vaccination,non-frail,62.2


In [3]:
df.drop(columns='Unnamed: 0', inplace=True)
df.head()

,SampleID,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000284735,ENSG00000284737,ENSG00000284738,ENSG00000284740,ENSG00000284743,ENSG00000284747,ENSG00000284748,SampleTiming,FrailtyStatus,Age
0,0029-0,7.0,31.0,7.0,385.0,3.0,12.0,30.0,39.0,3.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,pre-vaccination,frail,73.6
1,0029-3,7.0,37.0,10.0,400.0,3.0,15.0,34.0,46.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3 days post-vaccination,frail,73.6
2,0029-7,5.0,28.0,9.0,485.0,3.0,13.0,31.0,39.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7 days post-vaccination,frail,73.6
3,0104-0,11.0,47.0,10.0,320.0,4.0,14.0,30.0,63.0,3.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,pre-vaccination,non-frail,62.2
4,0104-3,9.0,36.0,10.0,203.0,3.0,13.0,47.0,56.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3 days post-vaccination,non-frail,62.2


In [4]:
counts = df.loc[:, 'ENSG00000000419':'ENSG00000284748']
meta = df[[
    "SampleID",
    "SampleTiming",
    "FrailtyStatus",
    "Age"
]]

In [5]:
# Split the data into frail and non-frail
f = df[df['FrailtyStatus'] == 'frail']
nf = df[df['FrailtyStatus'] == 'non-frail']

f_filter = f.loc[:, 'ENSG00000000419':'ENSG00000284748']
nf_filter = nf.loc[:, 'ENSG00000000419':'ENSG00000284748']

f_filter.head()

In [8]:
# Perform independent ttest
ttest_result = stats.ttest_ind(f_filter, nf_filter)
ttest_result

In [10]:
def add_ttest(df, ttest):
    
    # create new df with stats
    df = df.T
    stats_df = pd.DataFrame(columns=['tstat', 'pval'], index=df.index)

    # Add arrays as cols in a new dataframe with genes
    for ind, val in enumerate(ttest):
        if ind == 0:
            stats_df['tstat'] = val 
        else:
            stats_df['pval'] = val

    return stats_df

add_ttest(counts, ttest_result)

        

,tstat,pval
ENSG00000000419,1.159587,0.249582
ENSG00000000457,1.019352,0.311034
ENSG00000000460,1.266908,0.208775
ENSG00000000938,0.087204,0.930722
ENSG00000000971,3.903158,0.000194
...,...,...
ENSG00000284738,-1.079468,0.283544
ENSG00000284740,NaN,NaN
ENSG00000284743,NaN,NaN
ENSG00000284747,0.581515,0.562489


In [ ]:
# Perform fishers exact test between frail and non frail